<a href="https://colab.research.google.com/github/keripikkaneboo/Hands-On-Machine-Learning-O-Reilly-/blob/main/12.%20Chapter12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Bab 12: Costum Models and Training with TensorFlow

Meskipun API tingkat tinggi seperti `tf.keras` sudah sangat kuat, terkadang kita memerlukan kontrol lebih besar untuk mengimplementasikan arsitektur yang tidak biasa, *loss function* kustom, atau *training loop* yang spesifik. Bab ini membahas cara menggunakan API tingkat rendah TensorFlow untuk mendapatkan fleksibilitas tersebut.

* **TensorFlow Seperti NumPy**: Bab ini dimulai dengan menunjukkan bahwa TensorFlow pada dasarnya adalah library komputasi numerik yang sangat kuat, mirip dengan NumPy tetapi dengan dukungan GPU dan kemampuan untuk menghasilkan grafik komputasi.
    * ***Tensor***: Struktur data fundamental di TensorFlow, mirip dengan `ndarray` NumPy tetapi bersifat *immutable* (tidak dapat diubah).
    * ***Variable***: Digunakan untuk menyimpan parameter model yang dapat diubah (misalnya, bobot dan bias) selama training.
    * **Operasi**: TensorFlow menyediakan banyak sekali operasi matematika yang dapat dijalankan pada tensor.

* **Komponen Kustom di Keras**: Anda dapat mengkustomisasi hampir semua bagian dari Keras:
    * ***Loss Functions***: Anda bisa membuat fungsi Python sederhana atau membuat *class* yang mewarisi dari `keras.losses.Loss` jika Anda memerlukan *hyperparameter* atau *state*.
    * ***Metrics***: Serupa dengan *loss*, tetapi metrik digunakan untuk evaluasi. Untuk metrik yang nilainya perlu diakumulasi sepanjang *epoch* (seperti *precision*), Anda perlu membuat *class* yang mewarisi dari `keras.metrics.Metric` dan mengimplementasikan metode `update_state()` dan `result()`. Ini disebut **streaming metric**.
    * ***Layers***: Anda dapat membuat *layer* kustom dengan mewarisi dari `keras.layers.Layer` dan mengimplementasikan metode `build()` (untuk membuat bobot) dan `call()` (untuk logika *forward pass*).
    * ***Models***: Untuk arsitektur yang sangat kompleks dan dinamis (misalnya, yang berisi perulangan atau logika kondisional), Anda bisa membuat model kustom dengan mewarisi dari `keras.Model`.

* **Menghitung Gradien dengan Autodiff**:
    * **`tf.GradientTape`**: Alat utama TensorFlow untuk diferensiasi otomatis (*autodiff*). Ia merekam semua operasi yang melibatkan *variable* di dalam blok `with`, lalu dapat menghitung gradien dari sebuah *output* terhadap *variable-variable* tersebut.

* ***Custom Training Loop***:
    * Meskipun metode `fit()` sangat praktis, terkadang Anda perlu menulis *training loop* sendiri. Ini memberi Anda kontrol penuh atas proses training.
    * Langkah-langkahnya meliputi: iterasi manual melalui *epoch* dan *batch*, melakukan *forward pass* di dalam `tf.GradientTape`, menghitung *loss*, menghitung gradien, dan terakhir menerapkan gradien tersebut menggunakan optimizer.

* **TF Functions dan Graphs**:
    * ***`@tf.function`***: Sebuah *decorator* yang mengubah fungsi Python menjadi **TensorFlow Function (TF Function)** yang dapat dieksekusi dengan performa tinggi.
    * **AutoGraph dan Tracing**: Di balik layar, `@tf.function` menganalisis kode Python (*AutoGraph*) dan membuat grafik komputasi yang dioptimalkan untuk *input signature* tertentu (*tracing*). Hal ini membuat kode berjalan jauh lebih cepat dan portabel.

### 1. Custom Loss Function
Contoh membuat Huber loss sebagai sebuah *class* agar *hyperparameter* `threshold` dapat disimpan bersama model.

```python
from tensorflow import keras
import tensorflow as tf

class HuberLoss(keras.losses.Loss):
    def __init__(self, threshold=1.0, **kwargs):
        self.threshold = threshold
        super().__init__(**kwargs)

    def call(self, y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < self.threshold
        squared_loss = tf.square(error) / 2
        linear_loss  = self.threshold * tf.abs(error) - self.threshold**2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)

    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold": self.threshold}

# Contoh penggunaan:
# model.compile(loss=HuberLoss(2.0), optimizer="nadam")
```

### 2. Custom Layer
Contoh membuat lapisan `Dense` versi sederhana.

```python
class MyDense(keras.layers.Layer):
    def __init__(self, units, activation=None, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.activation = keras.activations.get(activation)

    def build(self, batch_input_shape):
        # Metode build() dipanggil saat pertama kali layer digunakan untuk membuat bobot
        self.kernel = self.add_weight(
            name="kernel", shape=[batch_input_shape[-1], self.units],
            initializer="glorot_normal")
        self.bias = self.add_weight(
            name="bias", shape=[self.units], initializer="zeros")
        super().build(batch_input_shape) # Wajib di akhir

    def call(self, X):
        # Logika forward pass
        return self.activation(X @ self.kernel + self.bias)
```

### 3. Menghitung Gradien dengan `tf.GradientTape`
Contoh sederhana untuk menghitung gradien dari sebuah fungsi.

```python
# Fungsi sederhana: f(w1, w2) = 3 * w1**2 + 2 * w1 * w2
def f(w1, w2):
    return 3 * w1 ** 2 + 2 * w1 * w2

w1, w2 = tf.Variable(5.), tf.Variable(3.)

with tf.GradientTape() as tape:
    z = f(w1, w2)

# Menghitung gradien z terhadap w1 dan w2
gradients = tape.gradient(z, [w1, w2])

print("Gradien:", gradients)
```

### 4. Custom Training Loop
Kerangka dasar untuk sebuah *custom training loop*.

```python
# 1. Membuat model sederhana (tanpa compile)
l2_reg = keras.regularizers.l2(0.05)
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="elu", kernel_initializer="he_normal",
                       kernel_regularizer=l2_reg),
    keras.layers.Dense(1, kernel_regularizer=l2_reg)
])

# 2. Menyiapkan data & hyperparameter (contoh dummy)
# Misalkan X_train_scaled dan y_train sudah disiapkan
# X_train_scaled, y_train = ...
n_epochs = 5
batch_size = 32
n_steps = len(X_train) // batch_size
optimizer = keras.optimizers.Nadam(learning_rate=0.01)
loss_fn = keras.losses.mean_squared_error

# 3. Training Loop
for epoch in range(1, n_epochs + 1):
    print(f"Epoch {epoch}/{n_epochs}")
    for step in range(1, n_steps + 1):
        # Ambil satu batch data
        # X_batch, y_batch = ...

        # Gunakan GradientTape untuk menghitung gradien
        # with tf.GradientTape() as tape:
        #     y_pred = model(X_batch, training=True)
        #     main_loss = tf.reduce_mean(loss_fn(y_batch, y_pred))
        #     # Menambahkan regularization loss dari model
        #     loss = tf.add_n([main_loss] + model.losses)

        # gradients = tape.gradient(loss, model.trainable_variables)
        
        # # Menerapkan gradien ke optimizer
        # optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        pass # Komentari pass dan un-comment kode di atas saat dijalankan

print("\nTraining selesai (kode training loop di-comment agar tidak error saat dijalankan tanpa data).")
```
**Catatan**: Kode *training loop* di atas hanya kerangka. Anda perlu menyediakan data (`X_train`, `y_train`, `X_batch`, `y_batch`) agar bisa berjalan penuh.

